In [1]:
%cd ~/ipython-notebook/dqn4wirelesscontrol/
import sys
import time
import pandas as pd
import lasagne
pd.set_option('mode.chained_assignment', None)  # block warnings due to DataFrame value assignment
# Project modules
from sleep_control.integration import Emulation
import sleep_control.traffic_emulator
import sleep_control.traffic_server
from sleep_control.controller import QController, DummyController
import qlearning.qtable
import qlearning.qnn
from qlearning.qtable import QAgent
from qlearning.qnn import QAgentNN
from qlearning.mixin import PhiMixin

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, CuDNN 4007)
/usr/local/lib/python2.7/dist-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


/home/admin-326/Data/ipython-notebook/dqn4wirelesscontrol


# Setting-up data

In [2]:
std_out = sys.stdout

In [3]:
session_df = pd.read_csv(filepath_or_buffer='./sleep_control/data/net_traffic_nonull.dat', 
                         sep=',', 
                         names=['uid','location','startTime_unix','duration_ms','domainProviders','domainTypes','domains','bytesByDomain','requestsByDomain'])
session_df.index.name = 'sessionID'
session_df['endTime_unix'] = session_df['startTime_unix'] + session_df['duration_ms']
session_df['startTime_datetime'] = pd.to_datetime(session_df['startTime_unix'], unit='ms')  # convert start time to readible date_time strings
session_df['endTime_datetime'] = pd.to_datetime(session_df['endTime_unix'], unit='ms')
session_df['totalBytes'] = session_df['bytesByDomain'].apply(lambda x: x.split(';')).map(lambda x: sum(map(float, x)))  # sum bytes across domains
session_df['totalRequests'] = session_df['requestsByDomain'].apply(lambda x: x.split(';')).map(lambda x: sum(map(float, x)))  # sum requests across domains
session_df.sort(['startTime_datetime'], ascending=True, inplace=True)  # get it sorted
session_df['interArrivalDuration_datetime'] = session_df.groupby('location')['startTime_datetime'].diff()  # group-wise diff
session_df['interArrivalDuration_ms'] = session_df.groupby('location')['startTime_unix'].diff()  # group-wise diff

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:10: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [4]:
class QAgentNNPhi(PhiMixin, QAgentNN):
    def __init__(self, **kwargs):
        super(QAgentNNPhi, self).__init__(**kwargs)

# EXP1 第三食堂

In [8]:
session_df_loc = session_df[session_df['location']=='第三食堂']
phi_length = 5
range_state_slice = [(0, 10), (0, 10), (0, 10), (0, 1), (0, 1)]
alpha, learning_rate = 0.9, 0.01
reward_scaling, reward_scaling_update = 20, 'fixed'
memory_size, num_buffer = 200, 2
freeze_period, batch_size = 10, 100
gamma=0.9
explore_strategy, epsilon = 'epsilon', 0.02
log_file = "sleep_control/experiments/log/message_2016-6-6_1345_Gamma20.log"
n_days=7

In [9]:
log_file = open(log_file,"w")
dim_state = (1, phi_length, 3+2)
actions = [(True, None), (False, 'serve_all')]
te = sleep_control.traffic_emulator.TrafficEmulator(session_df=session_df_loc, time_step=pd.Timedelta(seconds=2), verbose=2)
ts = sleep_control.traffic_server.TrafficServer(cost=5, verbose=2)
agent = QAgentNNPhi(phi_length=phi_length, dim_state=dim_state, range_state=[[range_state_slice]*phi_length], actions=actions,
        alpha=alpha, learning_rate=learning_rate, 
        reward_scaling=reward_scaling, reward_scaling_update=reward_scaling_update,
        memory_size=memory_size, num_buffer=num_buffer, freeze_period=freeze_period, batch_size=batch_size,
        gamma=gamma, explore_strategy=explore_strategy, epsilon=epsilon,
        verbose=2)
c = QController(agent=agent)
emu = sleep_control.integration.Emulation(te=te, ts=ts, c=c)
t = time.time()
sys.stdout = log_file
TOTAL_EPOCHS = 3600*24*n_days/2  # 7 days
while emu.epoch is not None and emu.epoch<TOTAL_EPOCHS:
    # log time
    print "Epoch {}, ".format(emu.epoch),
    left = emu.te.head_datetime + emu.te.epoch*emu.te.time_step
    right = left + emu.te.time_step
    print "{} - {}".format(left.strftime("%Y-%m-%d %H:%M:%S"), right.strftime("%Y-%m-%d %H:%M:%S"))
    emu.step()
    print
sys.stdout = std_out
log_file.close()
time.time()-t

New TrafficEmulator with parameters:
  head=2014-09-25 09:20:31.474000
  tail=2015-01-31 15:36:36.279000
  time_step=0 days 00:00:02
  epoch=0
  verbose=2


3977.4658041000366